In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

2024-08-05 00:55:32.254470: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 00:55:32.809689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 00:55:33.992531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Carrega a base de dados a partir de seu caminho
data = np.load('dataset.npz', allow_pickle=True)
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']

In [3]:
# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train, num_classes)

In [4]:
num_classes

5

In [5]:
#Model
initializer = keras.initializers.GlorotNormal(seed=12227)

inputs_layer = Input(shape=(224, 224, 3))
H = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs_layer)
H = MaxPooling2D(pool_size=(2, 2))(inputs_layer)
H = Conv2D(64, kernel_size=(3, 3), activation='relu')(H)
H=MaxPooling2D(pool_size=(2, 2))(H)
H=Conv2D(128, kernel_size=(3, 3), activation='relu')(H)
H=MaxPooling2D(pool_size=(2, 2))(H)
H=Conv2D(256, kernel_size=(3, 3), activation='relu')(H)
H=MaxPooling2D(pool_size=(2, 2))(H)
H=Flatten()(H)
H=Dense(256, activation='relu')(H)
H=Dense(256, activation='relu')(H)
H=Dropout(0.05)(H)
outputs_layer=Dense(num_classes, activation='softmax')(H)

model = keras.models.Model(inputs=inputs_layer, outputs=outputs_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2024-08-05 00:55:36.815506: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: UNKNOWN ERROR (100)


In [6]:
history = model.fit(X_train, y_train_categorical, epochs=100, batch_size=32)

Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 54s 368ms/step - accuracy: 0.4271 - loss: 33.6507
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 55s 387ms/step - accuracy: 0.5719 - loss: 0.9588
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 54s 380ms/step - accuracy: 0.6962 - loss: 0.7042
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 49s 346ms/step - accuracy: 0.7366 - loss: 0.6097
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 51s 340ms/step - accuracy: 0.7872 - loss: 0.5057
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 50s 352ms/step - accuracy: 0.8210 - loss: 0.4371
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 53s 371ms/step - accuracy: 0.8583 - loss: 0.3719
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 47s 332ms/step - accuracy: 0.9237 - loss: 0.2082
Epoch 9/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 49s 345ms/step - accuracy: 0.9531 - loss: 0.1339
Epoch 10/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 48s 336ms/step - accuracy: 0.9661 - loss: 0.1108
Epoch 11/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 49s 344ms/step - accuracy: 0.9700 - loss: 0.0897
Epoch 1

In [7]:
y_pred_prob = model.predict(X_test)

18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step


In [8]:
y_pred_classes = np.argmax(y_pred_prob, axis=1)
# Gera um DataFrame no formato esperado da submissão
num_samples = X_test.shape[0]
submission_df = pd.DataFrame({
    'ID': np.arange(1, num_samples + 1),
    'Prediction': y_pred_classes
})

# Salva o arquivo CSV no diretório atual
submission_df.to_csv('submission_task3_v2.csv', index=False)